# Split-Apply-Combine More

### Custom aggregate functions
Pandas groupby objects come with a few dozen aggregate functions that can be applied to the groups. These were printed out above. It is also possible to define your own customized aggregate function. These customized functions must return a single value.

Let's suppose you would like to know the difference between the max and min value of a column. Pandas does not have an aggregate function built to do this. You would have to define one yourself. 

Each customized aggregate function is defined as normal with the **`def`** keyword. Each function will accept one argument and that is the aggregating column. It will be passed as a **`Series`**. This means that all Series methods will work on the passed argument.

The **`min_max`** function below takes one argument, **`s`** which is a Series object. It returns the difference between the max and min values of that Series.

In [6]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 40

In [191]:
# define custom function
# s is a Series

def min_max(s):
    return s.max() - s.min()

In [192]:
# use new min_max function
college.groupby(['STABBR', 'RELAFFIL'])['UGDS'].agg(['size', 'min', 'max', min_max]).head(12)

size     min       max   min_max
STABBR RELAFFIL                                  
AK     0            7   109.0   12865.0   12756.0
       1            3    27.0     275.0     248.0
AL     0           72    12.0   29851.0   29839.0
       1           24    13.0    3033.0    3020.0
AR     0           68    18.0   21405.0   21387.0
       1           18    20.0    4485.0    4465.0
AS     0            1  1276.0    1276.0       0.0
AZ     0          124     1.0  151558.0  151557.0
       1            9    25.0    4102.0    4077.0
CA     0          609     0.0   44744.0   44744.0
       1          164     8.0    6745.0    6737.0
CO     0          118     0.0   25873.0   25873.0

### Filtering out groups

Groupby objects come with the **`filter`** method which is used to select only certain groups. The **`filter`** method accepts a function that returns either True or False. This result then filters the original DataFrame.

The function that **`filter`** accepts will be a **custom** function that accepts a single argument. In this case the arguemnt is a **DataFrame** of just one particular group. The function must return a **boolean** value, whether to keep the group or not.

The following function will be used to filter the college DataFrame for those states that have more than 300,000 total undergraduate students.

In [219]:
# This function accepts a dataframe of the current group
# it must return a boolean

def filter_ugds(df):
    return df['UGDS'].sum() > 300000

In [216]:
# use filter
college_filtered = college.groupby('STABBR').filter(filter_ugds)

In [220]:
# see the difference in size
print(college.shape)
print(college_filtered.shape)

(7535, 27)
(4619, 27)


### Groupby with Series

Pandas Series also have a **`groupby`** method. You might be thinking how it's possible to group and aggregate a single column of data. It becomes possible when you consider the **index** and the many levels that an index can have. Multi-level indexes will be discussed in another notebook.

To make the index meaningful you can set it to be one of the columns of the DataFrame with the **`set_index`** method.

In [780]:
# create a new dataframe with a more interesting index
college_state_index = college.set_index('STABBR')

college_state_index.head()

,INSTNM,CITY,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,UGDS_HISP,UGDS_ASIAN,UGDS_AIAN,UGDS_NHPI,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
STABBR,,,,,,,,,,,,,,,,,,,,,,,,,,
AL,Alabama A & M University,Normal,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,0.0055,0.0019,0.0024,0.0019,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300,33888
AL,University of Alabama at Birmingham,Birmingham,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,0.0283,0.0518,0.0022,0.0007,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700,21941.5
AL,Amridge University,Montgomery,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,0.0069,0.0034,0.0000,0.0000,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100,23370
AL,University of Alabama in Huntsville,Huntsville,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,0.6988,0.1255,0.0382,0.0376,0.0143,0.0002,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500,24097
AL,Alabama State University,Montgomery,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,0.0158,0.9208,0.0121,0.0019,0.0010,0.0006,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600,33118.5


### Grouping by the Index

The **`groupby`** method uses the **`level`** argument to contain the name (or integer location) of the index you would like to form groups with.

In [781]:
# create a Series by selecting one column
s = college_state_index['UGDS']

s.head()

STABBR
AL     4206.0
AL    11383.0
AL      291.0
AL     5451.0
AL     4811.0
Name: UGDS, dtype: float64

In [784]:
# the name can be retrieved with the name attribute of the index
s.index.name

'STABBR'

In [786]:
# groupby as normal but with the level argument
# get the mean of the undergrad student population

s.groupby(level='STABBR').mean().head(10)

STABBR
AK    2493.200000
AL    2789.865169
AR    1644.146341
AS    1276.000000
AZ    4130.468254
CA    3518.308397
CO    2324.880342
CT    1873.550562
DC    2645.277778
DE    2491.052632
Name: UGDS, dtype: float64

In [788]:
# should give the same result as dataframe
college.groupby('STABBR')['UGDS'].mean().head(10)

STABBR
AK    2493.200000
AL    2789.865169
AR    1644.146341
AS    1276.000000
AZ    4130.468254
CA    3518.308397
CO    2324.880342
CT    1873.550562
DC    2645.277778
DE    2491.052632
Name: UGDS, dtype: float64

### Tranforming and not aggregating groups

The most common operation to apply to a group is some kind of aggregation. Getting a single number summary of a group is usually of primary concern. There are however instances where the entire group would like to be transformed with every row kept. The **`transform`** groupby method will apply a (usually) custom function to each column of each group returning data that is the same length as the group. 

One of the most common operations is to standardize data - that is transform a numerical group so that its mean is 0 and standard deviation is 1. This is [done in the documentation](http://pandas.pydata.org/pandas-docs/stable/groupby.html#transformation). 

# Case Study: Tracking Weight Loss per Month
There are two friends that interested in tracking their weight over the course of several months. To provide motivation they decide to put some money on the line and wager some money each month. The friend who loses the highest percentage of body weight each month wins that month. Each month is independent from the others so the weight loss percentage resets at the start of the month.

Below is the (fake) data that was collected over the course of the bet.

In [848]:
# create fake data

all_weight = np.zeros(32)
all_weight[::2] = 300 * np.cumprod(np.random.rand(16) * .05 + .96)
all_weight[1::2] = 200 * np.cumprod(np.random.rand(16) * .05 + .96)
all_weight = np.round(all_weight, 0).astype(int)


df_weight = pd.DataFrame({'Name':['Bob', 'Amy'] * 16, 
                          'Month': ('Jan ' * 8 + ' Feb' * 8 + ' Mar' * 8 + ' Apr' * 8).split(),
                          'Week' : (['Week 1'] * 2 + ['Week 2'] * 2 + ['Week 3'] * 2 + ['Week 4'] * 2) * 4,
                          'Weight':all_weight},
                        columns=['Name', 'Month', 'Week', 'Weight'])

df_weight

,Name,Month,Week,Weight
0,Bob,Jan,Week 1,290
1,Amy,Jan,Week 1,196
2,Bob,Jan,Week 2,292
3,Amy,Jan,Week 2,189
4,Bob,Jan,Week 3,287
5,Amy,Jan,Week 3,183
6,Bob,Jan,Week 4,287
7,Amy,Jan,Week 4,177
8,Bob,Feb,Week 1,282
9,Amy,Feb,Week 1,177


### Transform
A new bet is begun at the start of each month and the winner for each month is declared by percentage weight loss achieved from week 1 to week 4. Percentage weight loss resets to zero at the beginning of each month.

We need to find a way to track the percentage weight loss within each month for each person. Since each week will have a weight loss percentage, no aggregation is performed and instead a new value is needed for each row. This situation calls for **`transform`**.

The default behavior of transform is to apply the same function (given as an argument) to each non-grouped column of the DataFrame. Since columns are Series, it is a Series that is passed to the transformation function.

In [861]:
# define a custom function that takes a Series and find the percentage loss from the first week
# is passed a Series

def find_perc_loss(s):
    return (s - s.iloc[0]) / s.iloc[0]

In [863]:
# apply transformation
# group by name and month
# and only transform weight column
df_weight['percent_month_loss'] = df_weight.groupby(['Name', 'Month'])['Weight'].transform(find_perc_loss) 

# view first two months. Notice that percent loss resets to 0
df_weight.head(16)

,Name,Month,Week,Weight,percent_month_loss
0,Bob,Jan,Week 1,290,0.000000
1,Amy,Jan,Week 1,196,0.000000
2,Bob,Jan,Week 2,292,0.006897
3,Amy,Jan,Week 2,189,-0.035714
4,Bob,Jan,Week 3,287,-0.010345
5,Amy,Jan,Week 3,183,-0.066327
6,Bob,Jan,Week 4,287,-0.010345
7,Amy,Jan,Week 4,177,-0.096939
8,Bob,Feb,Week 1,282,0.000000
9,Amy,Feb,Week 1,177,0.000000


In [864]:
# it might be easier to read if sorted.
# note that Month is not sorted lexicographically and not by calendar
df_weight_final = df_weight.sort_values(['Name', 'Month', 'Week'])

df_weight_final

,Name,Month,Week,Weight,percent_month_loss
25,Amy,Apr,Week 1,165,0.000000
27,Amy,Apr,Week 2,162,-0.018182
29,Amy,Apr,Week 3,157,-0.048485
31,Amy,Apr,Week 4,152,-0.078788
9,Amy,Feb,Week 1,177,0.000000
11,Amy,Feb,Week 2,171,-0.033898
13,Amy,Feb,Week 3,166,-0.062147
15,Amy,Feb,Week 4,166,-0.062147
1,Amy,Jan,Week 1,196,0.000000
3,Amy,Jan,Week 2,189,-0.035714


### Finding a winner

It's possible to manually find a winner of each month by comparing each person's week 4. For instance, Amy lost 7.9% of her body weight in April compared to Bob's 6.6% and won that month.

This is very tedious and since we have Pandas would be ridiculous to do by hand. We need to reshape the data in such a manner that each person's week 4 is easily comparable. There are many ways to reshape data. **`pivot`** and **`pivot_table`** DataFrame methods allow you to convert **long** formatted data into **wide** formatted data. This converts column values into column names. More will be said on this later.

First the above DataFrame will be filtered for only week 4 and then pivoted to make the comparison easy.

In [866]:
df_weight_week4 = df_weight_final[df_weight_final['Week'] == 'Week 4']

df_weight_week4

,Name,Month,Week,Weight,percent_month_loss
31,Amy,Apr,Week 4,152,-0.078788
15,Amy,Feb,Week 4,166,-0.062147
7,Amy,Jan,Week 4,177,-0.096939
23,Amy,Mar,Week 4,164,-0.012048
30,Bob,Apr,Week 4,228,-0.065574
14,Bob,Feb,Week 4,267,-0.053191
6,Bob,Jan,Week 4,287,-0.010345
22,Bob,Mar,Week 4,252,-0.063197


In [869]:
# use pivot_table to move the Name column
df_weight_winner = df_weight_week4.pivot_table(index='Month', columns='Name', values='percent_month_loss')

df_weight_winner

Name,Amy,Bob
Month,,
Apr,-0.078788,-0.065574
Feb,-0.062147,-0.053191
Jan,-0.096939,-0.010345
Mar,-0.012048,-0.063197


### Column for winner with np.where

Now that the winner is much more easily seen with the new reshaped data, a final step of creating a column with the winner's name can be made with the numpy **`where`** function. **`np.where`** works by an array of boolean values and returns an array consisting of the second argument wherever the array is True and the third argument False.

In [876]:
# a trivial example
# Return 'Yes' when True and 'No' when False

np.where([True, False, False, True, False, True], 'Yes', 'No')

array(['Yes', 'No', 'No', 'Yes', 'No', 'Yes'], 
      dtype='<U3')

In [873]:
# make the winner Amy when her weight loss is more than Bob's and vice versa using np.where
df_weight_winner['Winner'] = np.where(df_weight_winner['Amy'] < df_weight_winner['Bob'], 'Amy', 'Bob')

df_weight_winner

Name,Amy,Bob,Winner
Month,,,
Apr,-0.078788,-0.065574,Amy
Feb,-0.062147,-0.053191,Amy
Jan,-0.096939,-0.010345,Amy
Mar,-0.012048,-0.063197,Bob


In [878]:
# Get the winner
# If there happen to be lots of months of data

df_weight_winner['Winner'].value_counts()

Amy    3
Bob    1
Name: Winner, dtype: int64

# End of Section Summary
1. **`filter`** method returns a boolean for each group
2. Group by the index with the **`level`** argument in the **`groupby`** method
2. Know how to define custom aggregation and filter functions
3. Use **`transform`** when you want to return a Series the same length as the original group
4. Map a boolean expression to 2 other values with **`np.where`**

# Problem Set
We will be working with the city of Houston dataset for the questions in this notebook. Run the following command before attempting the problems

In [3]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 40
hou = pd.read_csv('data/coh_employee.csv')

In [4]:
hou.head()

,UNIQUE_ID,FIRST_NAME,LAST_NAME,POSITION_NUMBER,POSITION_JOB_CODE,POSITION_TITLE,BUSINESS_AREA,DEPARTMENT,PAY_GRADE,FUND_ID,FUND_NAME,COST_CENTER,COST_CENTER_NAME,REPORTS_TO_POSITION,MANAGER_NAME,BASE_SALARY,ORG_UNIT,ORG_UNIT_NAME,ETHNICITY,RACE,EMPLOYMENT_TYPE,EMPLOYMENT_SUB_GROUP,EXEMPT,GENDER,EMPLOYMENT_STATUS,CIVIL_SERVICE_TYPE,HIRE_DATE,JOB_DATE,COMP_DATE,EEOJ,SNAPSHOT_DATE
0,9172,LILLIAN,WARDEN,30027193,306.2,ASSISTANT DIRECTOR (EX LVL),1600,Municipal Courts Department,32,1000,General Fund,1600010001,MCD-Admin Services,30044747.0,NELLY SANTOS,121862.0,16.1410,MCD-ADMINISTRATIVE SERVICES,HISPANIC,Hispanic/Latino,Full Time,Exempt Excptn,E,Female,Active,CIVILIAN,2006-06-12,2012-10-13,1998-05-13,A: Officials & Administrators,2016-06-01
1,12311,ESPERANZA,RODRIGUEZ,30044399,901.2,LIBRARY ASSISTANT,3400,Library,05,1000,General Fund,3400070001,HPL-Neigh Lib Syst,30044159.0,MIGDALIA SEPULVEDA,26125.0,34.7001.27,HPL-NEIGHBORHOOD LIBRARY SERVICES,HISPANIC,Hispanic/Latino,Full Time,Non Exempt Postv,N,Female,Active,CIVILIAN,2000-07-19,2010-09-18,2000-07-19,E: Para-Professionals,2016-06-01
2,28693,JONATHAN,JORSCH JR.,30063393,108.0,POLICE OFFICER,1000,Houston Police Department-HPD,PA03,1000,General Fund,1000010068,HPD-Vehicular Crimes,30060941.0,CATRINA WILLIAMS,45279.0,10.1687.0130,HPD-VEHICULAR CRIMES DIVISION,"WHITE, NOT OF HISPANIC ORIGIN",White,Full Time,Non-Exempt Excptn,N,Male,Active,CLASSIFIED,2015-02-03,2015-02-03,2014-02-03,D: Protective Service Workers,2016-06-01
3,2359,JOHN,HARE,30007418,103.2,ENGINEER/OPERATOR,1200,Houston Fire Department (HFD),FD04,1000,General Fund,1200030001,HFD-Deployment,NaN,NaN,63166.0,12.1210,HFD-FIRE & EMS OPERATIONS,"WHITE, NOT OF HISPANIC ORIGIN",White,Full Time,NE Suprsn Excptn,N,Male,Active,CLASSIFIED,1982-02-08,1991-05-25,1982-02-08,D: Protective Service Workers,2016-06-01
4,5123,RUSSELL,GALBREATH,30006294,523.2,ELECTRICIAN,2500,General Services Department,18,2105,Maintenance Renewal and Replacement Fund,2500100003,GSD-MRR-Maint,30008673.0,JOHN BOGNEY,56347.0,25.1416,GSD-PROPERTY MGMT MAINT & OPER,"WHITE, NOT OF HISPANIC ORIGIN",White,Full Time,Non Exempt Postv,N,Male,Active,CIVILIAN,1989-06-19,1994-10-22,1989-06-19,G: Skilled Craft Workers,2016-06-01


### Problem 1
<span  style="color:green; font-size:16px">What are the 5 least common departments?</span>

In [5]:
g = hou.groupby('DEPARTMENT')
g['UNIQUE_ID'].count().sort_values().head()

DEPARTMENT
Convention and Entertainment     1
City Controller's Office         5
Mayor's Office                   5
Planning & Development           7
Houston Information Tech Svcs    9
Name: UNIQUE_ID, dtype: int64

### Problem 2
<span  style="color:green; font-size:16px">Filter out departments with less than 50 occurences and save it to **`hou_filter`**. Then test your code by outputing the frequencies of all the remaining departments. </span>

In [6]:
hou_filter = g.filter(lambda df: len(df) > 50)
hou_filter.groupby('DEPARTMENT')['UNIQUE_ID'].count().sort_values()

DEPARTMENT
Parks & Recreation                 74
Houston Airport System (HAS)      106
Health & Human Services           110
Public Works & Engineering-PWE    343
Houston Fire Department (HFD)     384
Houston Police Department-HPD     638
Name: UNIQUE_ID, dtype: int64

### Problem 3
<span  style="color:green; font-size:16px">Filter out departments from the original **`hou`** DataFrame with average salaries less than $70,000 and save it to **`hou_filter_salary`**. Then test your code by outputing the average salaries for the remaining departments.</span>

In [7]:
hou_filter_salary = g.filter(lambda df: df.BASE_SALARY.mean() > 70000)
hou_filter_salary.groupby('DEPARTMENT')['BASE_SALARY'].mean().sort_values(ascending=False)

DEPARTMENT
Legal Department                 104959.529412
Mayor's Office                    86489.600000
Finance                           79650.300000
Houston Information Tech Svcs     76112.777778
Name: BASE_SALARY, dtype: float64

### Problem 4
<span  style="color:green; font-size:16px">Filter for thoe departments from the original **`hou`** DataFrame with average salaries of at least 50,000 or having at least 5 unique position titles. Then create a way to check you got the right answer</span>

### Problem 5: Advanced
<span  style="color:green; font-size:16px">Find a way to do problem 4 without using the **`filter`** method. Make clever use of aggregate groupby and boolean logic</span>

In [8]:
def filter4(df):
    return (df.BASE_SALARY.mean() > 50000) | (len(df.POSITION_TITLE.unique()) >= 5)

In [9]:
hou_filter_4 = g.filter(filter4)

In [10]:
hou_filter_4.groupby('DEPARTMENT').agg({'BASE_SALARY':['mean'], 'POSITION_TITLE': ['count']})

,POSITION_TITLE,BASE_SALARY
,count,mean
DEPARTMENT,,
Admn. & Regulatory Affairs,29,50890.551724
City Controller's Office,5,55711.600000
City Council,11,59089.222222
Dept of Neighborhoods (DON),17,47092.882353
Finance,10,79650.300000
Fleet Management Department,36,43994.305556
General Services Department,22,51295.818182
Health & Human Services,110,51305.933962


### Problem 6: Advanced
<span  style="color:green; font-size:16px">Group by department, gender and race and get the mean, min and max base salary for each group. Also get the number of unique position titles and the most frequent position title for each group. Rename each aggregation to something that makes sense. Then remove the top level of the column index. Hint: This [stackoverflow answer](http://stackoverflow.com/questions/15222754/group-by-pandas-dataframe-and-select-most-common-string-factor) will be useful </span>

In [11]:
agg_dict = {'BASE_SALARY': {'SALARY_AVG': 'mean', 'SALARY_MIN': 'min', 'SALARY_MAX': 'max'}, 
            'POSITION_TITLE': {'UNIQUE_POSITIONS': 'count', 'MOST_FREQUENT_POSITION': lambda s: s.value_counts().index[0]}}

g = hou.groupby(['DEPARTMENT', 'GENDER', 'RACE']).agg(agg_dict)
g.columns = g.columns.droplevel(0)
g

MOST_FREQUENT_POSITION  \
DEPARTMENT                     GENDER RACE                                                                          
Admn. & Regulatory Affairs     Female Asian/Pacific Islander                    3-1-1 TELECOMMUNICATOR SUPERVISOR   
                                      Black or African American                 CUSTOMER SERVICE REPRESENTATIVE I   
                                      Hispanic/Latino                                      ANIMAL CARE TECHNICIAN   
                                      White                                             ADMINISTRATIVE SPECIALIST   
                               Male   Black or African American                            ANIMAL CARE TECHNICIAN   
                                      Hispanic/Latino                           CUSTOMER SERVICE REPRESENTATIVE I   
                                      White                                     DEPUTY DIRECTOR (EXECUTIVE LEVEL)   
City Controller's Office       Female Asian/Pacific Islander                        ASSISTANT CITY CONTROLLER III   
                                      Black or African American                          ADMINISTRATIVE ASSISTANT   
                                      Hispanic/Latino                                    ADMINISTRATIVE ASSISTANT   
                               Male   Black or African American                               FINANCIAL ANALYST I   
City Council                   Female Black or African American             SENIOR COUNCIL AIDE (EXECUTIVE LEVEL)   
                                      Hispanic/Latino                         COUNCIL SECRETARY (EXECUTIVE LEVEL)   
                                      White                                                        COUNCIL MEMBER   
                               Male   Black or African American          COUNCIL RESEARCH ASSISTANT (EXECUTIVE LE   
                                      Hispanic/Latino                            COUNCIL INTERN (EXECUTIVE LEVEL)   
                                      White                                                        COUNCIL MEMBER   
Convention and Entertainment   Female Hispanic/Latino                                      MAINTENANCE SUPERVISOR   
Dept of Neighborhoods (DON)    Female Black or African American                        CODE ENFORCEMENT OFFICER I   
                                      Hispanic/Latino                                    SENIOR COMMUNITY LIAISON   
                                      White                              ADMINISTRATIVE COORDINATOR (EXECUTIVE LE   
                               Male   American Indian or Alaskan Native                                   LABORER   
                                      Black or African American                       CODE ENFORCEMENT OFFICER II   
                                      Hispanic/Latino                                           INSPECTOR TRAINEE   
Finance                        Female Black or African American                              FINANCIAL ANALYST IV   
                                      Hispanic/Latino                                               STAFF ANALYST   
                               Male   Black or African American                              SENIOR STAFF ANALYST   
                                      Hispanic/Latino                                      PROCUREMENT SPECIALIST   
Fleet Management Department    Female Black or African American                         ADMINISTRATIVE SPECIALIST   
                               Male   Asian/Pacific Islander                                         MECHANIC III   
...                                                                                                           ...   
Municipal Courts Department    Male   Asian/Pacific Islander                  ASSOCIATE JUDGE OF MUNICIPAL COURTS   
                                      Black or African American                 MUNICIPAL COURTS SECURITY OFFICER

### Problem 7
<span  style="color:green; font-size:16px"> Create a column **`is_max`** that is equal to 1 if the base salary is currently the max base salary (out of all previous rows) for that department and 0 otherwise. Save the returned DataFrame to **`hou_1`**. Output the first 20 rows. See sample data below.</span>

In [12]:
# Return a dataframe that looks like this
''' 
DEPARTMENT    BASE_SALARY  is_max  
Library           160         1
Police            150         1
Library           170         1
Police             95         0
Police            140         0
Library            80         0
Police            189         1
'''

' \nDEPARTMENT    BASE_SALARY  is_max  \nLibrary           160         1\nPolice            150         1\nLibrary           170         1\nPolice             95         0\nPolice            140         0\nLibrary            80         0\nPolice            189         1\n'

In [13]:
def is_max(s):
    return np.where(s == s.cummax(), 1, 0)

In [14]:
# for simplicity start with this dataframe
hou_1 = hou[['DEPARTMENT', 'BASE_SALARY']].copy() # copy so that we don't overwrite the original data
g1 = hou_1.groupby('DEPARTMENT')
hou_1['is_max'] = g1.BASE_SALARY.transform(is_max)
hou_1
# for n, g in g1:
#     print(g)

,DEPARTMENT,BASE_SALARY,is_max
0,Municipal Courts Department,121862.0,1.0
1,Library,26125.0,1.0
2,Houston Police Department-HPD,45279.0,1.0
3,Houston Fire Department (HFD),63166.0,1.0
4,General Services Department,56347.0,1.0
5,Houston Police Department-HPD,66614.0,1.0
6,Public Works & Engineering-PWE,71680.0,1.0
7,Houston Airport System (HAS),42390.0,1.0
8,Public Works & Engineering-PWE,107962.0,1.0
9,Houston Airport System (HAS),44616.0,1.0


### Problem 8: Advanced
<span  style="color:green; font-size:16px"> Programatically Find the 10th occurence of 0 for **`is_max`** and return a DataFrame that ends after the tenth occurence.</span>

In [15]:
occ_10_is_max = hou_1.loc[hou_1.is_max==0, :].index[9]
hou_1.loc[:occ_10_is_max,:]

,DEPARTMENT,BASE_SALARY,is_max
0,Municipal Courts Department,121862.0,1.0
1,Library,26125.0,1.0
2,Houston Police Department-HPD,45279.0,1.0
3,Houston Fire Department (HFD),63166.0,1.0
4,General Services Department,56347.0,1.0
5,Houston Police Department-HPD,66614.0,1.0
6,Public Works & Engineering-PWE,71680.0,1.0
7,Houston Airport System (HAS),42390.0,1.0
8,Public Works & Engineering-PWE,107962.0,1.0
9,Houston Airport System (HAS),44616.0,1.0


### Problem 9
<span  style="color:green; font-size:16px"> Write a function that accepts a single argument that will filter **`hou_1`** for a specific department where **`is_max`** is 1. Test your function with departments like 'Library' and 'Public Works & Engineering-PWE'.</span>

In [16]:
def is_max_1(df):
    return df[df.is_max == 1]

In [20]:
is_max_1(hou_1[hou_1.DEPARTMENT=='Library'])

,DEPARTMENT,BASE_SALARY,is_max
1,Library,26125.0,1.0
280,Library,31034.0,1.0
298,Library,38563.0,1.0
308,Library,49317.0,1.0
412,Library,79302.0,1.0
1165,Library,107763.0,1.0


### Problem 10
<span  style="color:green; font-size:16px">A good skill to have is to ask a difficult question for yourself and then answer it. Ask yourself a question that involes grouping and answer it.</span>